In [63]:
import pandas as pd
import numpy as np
import dask.array as da
import dask.dataframe as dd
import dask.delayed as delayed
import time
import math
#import graphviz
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import h5py
import dask

In [64]:
def read_filelist(loc_dir,prefix,unie,fileformat):
    # Read the filelist in the specific directory
    str = os.popen("ls "+ loc_dir + prefix + unie + "*."+fileformat).read()
    fname = np.array(str.split("\n"))
    fname = np.delete(fname,len(fname)-1)
    return fname

def read_MODIS(fname1,fname2,verbose=False): # READ THE HDF FILE
    # Read the cloud mask from MYD06_L2 product') 
    ncfile=Dataset(fname1,'r')
    CM1km = np.array(ncfile.variables['Cloud_Mask_1km'])
    CM   = (np.array(CM1km[:,:,0],dtype='byte') & 0b00000110) >>1
    ncfile.close()
    
    ncfile=Dataset(fname2,'r')
    lat  = np.array(ncfile.variables['Latitude'])
    lon  = np.array(ncfile.variables['Longitude'])
    attr_lat = ncfile.variables['Latitude']._FillValue
    attr_lon = ncfile.variables['Longitude']._FillValue
    return lat,lon,CM

In [65]:


def countzero(x, axis=1):
    #print(x)
    count0 = 0
    count1 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    #print(count0/len(x))
    return (count0/len(x))

def results(concat_list):
    #b2=dd.concat(b1)
    #b_2=b2.groupby(['Latitude','Longitude']).mean().reset_index()
    #b3=b_2.reset_index()
    combs=[]
    for x in range(0,180):
        for y in range(0,360):
            combs.append((x, y))
    df_1=pd.DataFrame(combs)
    df_1.columns=['Latitude','Longitude']
    df4=pd.merge(df_1, df2,on=('Latitude','Longitude'), how='left')
    df_cm=df4['CM'].values
    np_cm=df_cm.reshape(180,360)
    return np_cm

In [67]:
MYD06_dir= '/Users/dprakas1/Desktop/modis_one/'
MYD06_prefix = 'MYD06_L2.A2008'
MYD03_dir= '/Users/dprakas1/Desktop/modis_one/'
MYD03_prefix = 'MYD03.A2008'
fileformat = 'hdf'

fname1,fname2 = [],[]


days = np.arange(1,31,dtype=np.int)
for day in days:
    dc ='%03i' % day
    fname_tmp1 = read_filelist(MYD06_dir,MYD06_prefix,dc,fileformat)
    fname_tmp2 = read_filelist(MYD03_dir,MYD03_prefix,dc,fileformat)
    fname1 = np.append(fname1,fname_tmp1)
    fname2 = np.append(fname2,fname_tmp2)

# Initiate the number of day and total cloud fraction
files  = np.arange(len(fname1))



for j in range(0,1):#hdfs:
    print('steps: ',j+1,'/ ',(fname1)) 

    # Read Level-2 MODIS data
    lat,lon,CM = read_MODIS(fname1[j],fname2[j])
print((fname2))
print((fname1))
#rint(CM)
#lat = lat.ravel()
#lon = lon.ravel()
#CM  = CM.ravel()
CM.shape



steps:  1 /  ['/Users/dprakas1/Desktop/modis_one/MYD06_L2.A2008001.0005.006.2013341193207.hdf']
['/Users/dprakas1/Desktop/modis_one/MYD03.A2008001.0005.006.2012066122516.hdf']
['/Users/dprakas1/Desktop/modis_one/MYD06_L2.A2008001.0005.006.2013341193207.hdf']


(2030, 1354)

In [68]:
combs=[]
for x in range(0,180):
    for y in range(0,360):
        combs.append((x, y))
df_1=pd.DataFrame(combs)
df_1.columns=['Latitude','Longitude']
   

In [69]:
%%time
b1=[]
def aggregateOneFileData(M06_file, M03_file):
    
    cm = np.zeros((2030,1354), dtype=np.float32)
    lat = np.zeros((2030,1354), dtype=np.float32)
    lon = np.zeros((2030,1354), dtype=np.float32)
    
    #print(fname1,fname2)
    myd06 = Dataset(M06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    
    #print("CM intial shape:",CM.shape)
    cm = da.concatenate((cm,CM),axis=0)
    #print("CM shape after con:",cm.shape)
    cm=da.ravel(cm)
    #print("cm shape after ravel:",cm.shape)
    myd03 = Dataset(M03_file, "r")
    latitude = myd03.variables["Latitude"][:,:]
    longitude = myd03.variables["Longitude"][:,:]
    #print("Lat intial shape:",latitude.shape)
    #print("lon intial shape:",longitude.shape)
    
    lat = da.concatenate((lat,latitude),axis=0)
    lon = da.concatenate((lon,longitude),axis=0)
    #print("lat shape after con:",lat.shape)
    #print("lon shape after con:",lon.shape)
    
    lat=da.ravel(lat)
    lon=da.ravel(lon)
    
    #print("lat shape after ravel:",lat.shape)
    #print("lon shape after ravel:",lon.shape)
    cm=cm.astype(int)
    lon=lon.astype(int)
    lat=lat.astype(int)
    lat=lat+90
    lon=lon+180
    Lat=lat.to_dask_dataframe()
    Lon=lon.to_dask_dataframe()
    CM=cm.to_dask_dataframe()
    df=dd.concat([Lat,Lon,CM],axis=1,interleave_partitions=False)
    print(type(df))
    
    cols = {0:'Latitude',1:'Longitude',2:'CM'}
    df = df.rename(columns=cols)
    
    df2=(df.groupby(['Longitude','Latitude']).CM.apply(countzero).reset_index())
    df3=df2.compute()
    df4=pd.merge(df_1, df3,on=('Latitude','Longitude'), how='left')
    df_cm=df4['CM'].values
    np_cm=df_cm.reshape(180,360)
    return np_cm

CPU times: user 29 µs, sys: 17 µs, total: 46 µs
Wall time: 49.8 µs


In [70]:
tt=aggregateOneFileData(fname1[0],fname2[0])

<class 'dask.dataframe.core.DataFrame'>


/Users/dprakas1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


In [71]:
tt

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [72]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

/Users/dprakas1/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [73]:
result=client.map(aggregateOneFileData,fname1[0],fname2[0])

In [74]:
result

[<Future: status: error, key: aggregateOneFileData-97d3907dc27491a1a4516e7ae69caa0d>,
 <Future: status: error, key: aggregateOneFileData-ab0c88413c49ca923cdc9fb889b34937>,
 <Future: status: error, key: aggregateOneFileData-3f1c51c4a70e5549859f394151aa7373>,
 <Future: status: error, key: aggregateOneFileData-8fef3ce56bcb090b5addc18537b984ab>,
 <Future: status: error, key: aggregateOneFileData-0cd5ce33a09d03256df02adc2d9a33f0>,
 <Future: status: error, key: aggregateOneFileData-3f1c51c4a70e5549859f394151aa7373>,
 <Future: status: error, key: aggregateOneFileData-97d3907dc27491a1a4516e7ae69caa0d>,
 <Future: status: error, key: aggregateOneFileData-eee3b68f12414c136be29ef7760467ae>,
 <Future: status: error, key: aggregateOneFileData-4deebe83d5d55f8a0c94122a64e5f0d7>,
 <Future: status: error, key: aggregateOneFileData-0cd5ce33a09d03256df02adc2d9a33f0>,
 <Future: status: error, key: aggregateOneFileData-692755c8249735916a13882c2a4fe52a>,
 <Future: status: error, key: aggregateOneFileData-df6

In [37]:
import pandas as pd
import numpy as np
import dask.array as da
import dask.dataframe as dd
import dask.delayed as delayed
import time
import math
#import graphviz
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import h5py
import dask
import glob


In [62]:
M03_dir = "/Users/dprakas1/Desktop/modis_one/MYD03/"
M06_dir = "/Users/dprakas1/Desktop/modis_one/MYD06_L2/"
M03_files = sorted(glob.glob(M03_dir + "MYD03.A2008*"))
M06_files = sorted(glob.glob(M06_dir + "MYD06_L2.A2008*"))


In [63]:
myd06 = Dataset(M06_files, "r")
CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
CM = np.array(CM).byteswap().newbyteorder()
    

FileNotFoundError: [Errno 2] No such file or directory: b'[]'

In [64]:
MOD03_path =sys.argv[1] #'HDFFiles/'
MOD03_path = '/Users/dprakas1/Desktop/modis_one/'
MOD06_path =sys.argv[2] #'HDFFiles/'
MOD06_path = '/Users/dprakas1/Desktop/modis_one/'

In [65]:
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    #print('Reading The Cloud Mask From MOD06_L2 Product:')
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
   # print('The Level-2 Cloud Mask Array Shape',CM.shape)
    print(' ')

    myd03 = Dataset(MOD03_file, "r")
    #print('Reading The Latitude-Longitude From MOD03 Product:')
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #print('The Level-2 Latitude-Longitude Array Shape',latitude.shape)
    print(' ')

    return latitude,longitude,CM



In [66]:
MOD03_filepath = 'MYD03.A*.hdf'
MOD06_filepath = 'MYD06_L2.A*.hdf'
MOD03_filename, MOD06_filename =[],[]
MOD03_filename2, MOD06_filename2 =[],[]

for MOD06_filelist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_filelist, MOD06_filepath):
        MOD06_filename = MOD06_filelist
        MOD06_filename2.append(MOD06_filelist)

for MOD03_filelist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_filelist, MOD03_filepath):
        MOD03_filename = MOD03_filelist
        MOD03_filename2.append(MOD03_filelist)

if MOD03_filename and MOD06_filename:
    print('Reading Level 2 GeoLocation & Cloud Data')
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_filename,MOD03_path+MOD03_filename)

print('The Number Of Files In The MODO3 List: ')
print(len(MOD03_filename2))
print(' ')
print('The Number Of Files In The MODO6_L2 List: ')
print(len(MOD06_filename2))
print(' ')

Reading Level 2 GeoLocation & Cloud Data
 
 
The Number Of Files In The MODO3 List: 
1
 
The Number Of Files In The MODO6_L2 List: 
1
 


In [68]:
MOD03_filename2

['MYD03.A2008001.0005.006.2012066122516.hdf']

In [73]:
result=aggregateOneFileData(MOD06_filename2, MOD03_filename2)

FileNotFoundError: [Errno 2] No such file or directory: b"['MYD06_L2.A2008001.0005.006.2013341193207.hdf']"

In [72]:
%%time
b1=[]
def aggregateOneFileData(MOD06_filename2, MOD03_filename2):
    
    cm = np.zeros((2030,1354), dtype=np.float32)
    lat = np.zeros((2030,1354), dtype=np.float32)
    lon = np.zeros((2030,1354), dtype=np.float32)
    
    #print(fname1,fname2)
    myd06 = Dataset(MOD06_filename2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    
    #print("CM intial shape:",CM.shape)
    cm = da.concatenate((cm,CM),axis=0)
    #print("CM shape after con:",cm.shape)
    cm=da.ravel(cm)
    #print("cm shape after ravel:",cm.shape)
    myd03 = Dataset(MOD03_filename2, "r")
    latitude = myd03.variables["Latitude"][:,:]
    longitude = myd03.variables["Longitude"][:,:]
    #print("Lat intial shape:",latitude.shape)
    #print("lon intial shape:",longitude.shape)
    
    lat = da.concatenate((lat,latitude),axis=0)
    lon = da.concatenate((lon,longitude),axis=0)
    #print("lat shape after con:",lat.shape)
    #print("lon shape after con:",lon.shape)
    
    lat=da.ravel(lat)
    lon=da.ravel(lon)
    
    #print("lat shape after ravel:",lat.shape)
    #print("lon shape after ravel:",lon.shape)
    cm=cm.astype(int)
    lon=lon.astype(int)
    lat=lat.astype(int)
    lat=lat+90
    lon=lon+180
    Lat=lat.to_dask_dataframe()
    Lon=lon.to_dask_dataframe()
    CM=cm.to_dask_dataframe()
    df=dd.concat([Lat,Lon,CM],axis=1,interleave_partitions=False)
    print(type(df))
    
    cols = {0:'Latitude',1:'Longitude',2:'CM'}
    df = df.rename(columns=cols)
    
    df2=(df.groupby(['Longitude','Latitude']).CM.apply(countzero).reset_index())
    print(type(df2))
    combs=[]
    for x in range(0,180):
        for y in range(0,360):
            combs.append((x, y))
    df_1=pd.DataFrame(combs)
    df_1.columns=['Latitude','Longitude']
    df_2=dd.from_pandas(df_1,npartitions=10500)
    df4=dd.merge(df_2, b_2,on=('Latitude','Longitude'), how='left')
    a = df4['CM'].to_dask_array(lengths=True)
    arr = da.asarray(a, chunks=(9257))
    final_array=arr.reshape(180,360).compute()
    return final_array

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 5.96 µs


In [ ]:
myd06_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
def aggregateOneFileData(M06_file, M03_file):
    
    cm = np.zeros((2030,1354), dtype=np.float32)
    lat = np.zeros((2030,1354), dtype=np.float32)
    lon = np.zeros((2030,1354), dtype=np.float32)
    
    cmfilelist = []

    for MOD06_file in MOD06_filename2:
        MOD06_file2 = myd06_name + MOD06_file
        myd06 = Dataset(MOD06_file2, "r")
        CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
        CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
        CM = np.array(CM).byteswap().newbyteorder()

        #----#
        cm = da.from_array(CM, chunks =(2030,1354))
        #cm = da.from_array(CM, chunks =(500,500))
        cmfilelist.append(cm)
        cm = da.stack(cmfilelist, axis=0)


    print('The Cloud Mask Array Shape Is: ',cm.shape)


    #print("CM intial shape:",CM.shape)
    cm = da.concatenate((cm,CM),axis=0)
    #print("CM shape after con:",cm.shape)
    cm=da.ravel(cm)
    #print("cm shape after ravel:",cm.shape)
    myd03 = Dataset(fname2, "r")
    latitude = myd03.variables["Latitude"][:,:]
    longitude = myd03.variables["Longitude"][:,:]
    #print("Lat intial shape:",latitude.shape)
    #print("lon intial shape:",longitude.shape)
    
    lat = da.concatenate((lat,latitude),axis=0)
    lon = da.concatenate((lon,longitude),axis=0)
    #print("lat shape after con:",lat.shape)
    #print("lon shape after con:",lon.shape)
    
    lat=da.ravel(lat)
    lon=da.ravel(lon)
    
    #print("lat shape after ravel:",lat.shape)
    #print("lon shape after ravel:",lon.shape)
    cm=cm.astype(int)
    lon=lon.astype(int)
    lat=lat.astype(int)
    lat=lat+90
    lon=lon+180
    Lat=lat.to_dask_dataframe()
    Lon=lon.to_dask_dataframe()
    CM=cm.to_dask_dataframe()
    df=dd.concat([Lat,Lon,CM],axis=1,interleave_partitions=False)
    print(type(df))
    
    cols = {0:'Latitude',1:'Longitude',2:'CM'}
    df = df.rename(columns=cols)
    
    df2=(df.groupby(['Longitude','Latitude']).CM.apply(countzero).reset_index())
    print(type(df2))
    combs=[]
    for x in range(0,180):
        for y in range(0,360):
            combs.append((x, y))
    df_1=pd.DataFrame(combs)
    df_1.columns=['Latitude','Longitude']
    df_2=dd.from_pandas(df_1,npartitions=10500)
    df4=dd.merge(df_2, b_2,on=('Latitude','Longitude'), how='left')
    a = df4['CM'].to_dask_array(lengths=True)
    arr = da.asarray(a, chunks=(9257))
    final_array=arr.reshape(180,360).compute()
    return final_array